In [1]:
import pandas as pd
import numpy as np

In [2]:
user = pd.read_csv("data/Users.csv", encoding="latin1", engine="python", on_bad_lines="skip")
book = pd.read_csv("data/Books.csv", encoding='latin1')
rating=pd.read_csv("data/Ratings.csv",encoding='latin1')

C:\Users\siddh\AppData\Local\Temp\ipykernel_4316\2766897497.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv("data/Books.csv", encoding='latin1')


In [3]:
user.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [4]:
book.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
rating.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
rating_up = rating.merge(book[['ISBN', 'Book-Title']], on='ISBN', how='left')
rating_up

,User-ID,ISBN,Book-Rating,Book-Title
0,276725,034545104X,0,Flesh Tones: A Novel
1,276726,0155061224,5,Rites of Passage
2,276727,0446520802,0,The Notebook
3,276729,052165615X,3,Help!: Level 1
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...
...,...,...,...,...
1149775,276704,1563526298,9,Get Clark Smart : The Ultimate Guide for the S...
1149776,276706,0679447156,0,Eight Weeks to Optimum Health: A Proven Progra...
1149777,276709,0515107662,10,The Sherbrooke Bride (Bride Trilogy (Paperback))
1149778,276721,0590442449,10,Fourth Grade Rats


In [7]:
null_values1 = rating_up['Book-Title'].isnull().sum()
null_values2 = len(rating_up)

In [8]:
print(f'These are the number of non null records : {null_values2 - null_values1}')

These are the number of non null records : 1031136


In [9]:
rating_up.dropna(subset=['Book-Title'],inplace=True)
rating_up.head()

,User-ID,ISBN,Book-Rating,Book-Title
0,276725,034545104X,0,Flesh Tones: A Novel
1,276726,0155061224,5,Rites of Passage
2,276727,0446520802,0,The Notebook
3,276729,052165615X,3,Help!: Level 1
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...


In [10]:
len(rating_up)

1031136

In [11]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [12]:
rating_up['user_index'] = rating_up['User-ID'].astype('category').cat.codes
rating_up['book_index'] = rating_up['Book-Title'].astype('category').cat.codes

In [13]:
user_book_matrix = csr_matrix(
    (rating_up['Book-Rating'], (rating_up['user_index'], rating_up['book_index']))
)
print(f"User-Book Matrix Shape: {user_book_matrix.shape}")

User-Book Matrix Shape: (92106, 241071)


In [15]:
from sklearn.neighbors import NearestNeighbors 

user_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1) 
user_knn.fit(user_book_matrix) 

distances, indices = user_knn.kneighbors(user_book_matrix[3], n_neighbors=5) 
print(indices) 
print(distances) 

[[92088 92089 92090 92091 92092]]
[[1. 1. 1. 1. 1.]]


In [17]:
user_id_mapping = dict(enumerate(rating_up['User-ID'].astype('category').cat.categories)) 
book_title_mapping = dict(enumerate(rating_up['Book-Title'].astype('category').cat.categories))

similar_users = [user_id_mapping[i] for i in indices.flatten()]

In [18]:
# Example: get 10 most similar users to user_index 0 
target_user_index = 0
target_user_id = user_id_mapping[target_user_index]

similar_users_books = rating_up[rating_up['User-ID'].isin(similar_users)] 
target_user_books = set(rating_up[rating_up['User-ID'] == target_user_id]['Book-Title']) 

In [19]:
# Step 6: Recommend books that similar users liked (say rating ≥ 8) 
recommended_books = ( similar_users_books[
                     similar_users_books['Book-Rating'] >= 5]
                     .loc[~similar_users_books['Book-Title']
                     .isin(target_user_books)]
                     .groupby('Book-Title')['Book-Rating']
                     .mean().sort_values(ascending=False).head(10)) 
print(f"Top recommendations for User {target_user_id}:") 
print(recommended_books) 

Top recommendations for User 2:
Book-Title
Wringer (Trophy Newbery)                                   10.0
She Said Yes : The Unlikely Martyrdom of Cassie Bernall     9.0
The Summerhouse                                             6.0
Name: Book-Rating, dtype: float64


In [20]:
book_user_matrix = csr_matrix( (rating_up['Book-Rating'], (rating_up['book_index'], rating_up['user_index'])) ) 
print(book_user_matrix.shape)

(241071, 92106)


In [21]:
book_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6, n_jobs=-1) 
book_knn.fit(book_user_matrix) 

# Example: get similar books to the first book (index 0) 
distances, indices = book_knn.kneighbors(book_user_matrix[0], n_neighbors=6) 
print(indices) 
print(distances) 

[[     0 239812 239807 231273 233964 211442]]
[[0. 0. 0. 0. 0. 0.]]


In [22]:
book_mapping = dict( zip(rating_up['book_index'], rating_up['Book-Title']) ) 
book_id = 0 # example book index 

similar_books = indices[0][1:] 
# exclude itself 

print("Target Book:", book_mapping[book_id]) 
print("Similar Books:") 
for idx in similar_books: 
    print("-", book_mapping[idx])

Target Book:  A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)
Similar Books:
- Young Indiana Jones and the Princess of Peril
- Young Indiana Jones &amp; the Gypsy Revenge
- Wendy and the Bullies
- Whitney Cousins: Heather (Whitney Cousins)
- The Ugly Duckling (Fairy Tale Classics Storybook)


In [23]:
user_index_mapping = {v: k for k, v in user_id_mapping.items()}
book_index_mapping = {v: k for k, v in book_mapping.items()}

In [29]:
def hybrid_recommend(target_user_id, rating_df, top_n=10):
    # --- Step A: Find target user index ---
    if target_user_id not in user_index_mapping:
        print("User not found.")
        return
    
    target_user_index = user_index_mapping[target_user_id]

    # --- Step B: Find similar users ---
    distances, indices = user_knn.kneighbors(user_book_matrix[target_user_index], n_neighbors=6)
    similar_users = [user_id_mapping[i] for i in indices.flatten() if i != target_user_index]
    print(f"\n👥 Similar Users to {target_user_id}: {similar_users}")

    # --- Step C: Books read and rated by target user ---
    target_user_data = rating_df[rating_df['User-ID'] == target_user_id][['Book-Title', 'Book-Rating']]
    target_user_books = set(target_user_data['Book-Title'])

    print(f"\n📖 Books read by User {target_user_id}:")
    if target_user_data.empty:
        print("This user has not rated any books yet.")
    else:
        for _, row in target_user_data.iterrows():
            print(f"- {row['Book-Title']} (Rating: {row['Book-Rating']})")

    # --- Step D: Books liked by similar users (User–User based CF) ---
    similar_users_books = rating_df[rating_df['User-ID'].isin(similar_users)]
    recommended_books_user = (
        similar_users_books[similar_users_books['Book-Rating'] >= 5]
        .loc[~similar_users_books['Book-Title'].isin(target_user_books)]
        .groupby('Book-Title')['Book-Rating']
        .mean()
        .sort_values(ascending=False)
        .head(top_n)
    )

    print("\n📘 Top recommendations from similar users (User–User CF):")
    if recommended_books_user.empty:
        print("No new books found from similar users.")
    else:
        for book, rating in recommended_books_user.items():
            print(f"- {book} (avg rating: {rating:.2f})")

    # --- Step E: Books similar to what target user read (Item–Item based CF) ---
    similar_books_set = set()
    for book in target_user_books:
        if book in book_index_mapping:
            book_id = book_index_mapping[book]
            distances, indices = book_knn.kneighbors(book_user_matrix[book_id], n_neighbors=4)
            for idx in indices.flatten()[1:]:
                similar_books_set.add(book_mapping[idx])

    print("\n📙 Books similar to what the user has read (Item–Item CF):")
    if not similar_books_set:
        print("No similar books found.")
    else:
        similar_books_ratings = (
            rating_df[rating_df['Book-Title'].isin(similar_books_set)]
            .groupby('Book-Title')['Book-Rating']
            .mean()
            .sort_values(ascending=False)
        )

        for book, avg_rating in similar_books_ratings.head(top_n).items():
            print(f"- {book} (avg rating: {avg_rating:.2f})")

    # --- Step F: Combine both sources ---
    final_recommendations = set(recommended_books_user.index).union(similar_books_set)

    print(f"\n📚 Final Combined Recommendations for User {target_user_id}:")
    if not final_recommendations:
        print("No recommendations available.")
    else:
        for book in list(final_recommendations)[:top_n]:
            print("-", book)


In [31]:
hybrid_recommend(target_user_id=276709, rating_df=rating_up, top_n=10)


👥 Similar Users to 276709: [157116, 170845, 160878, 45064, 112559]

📖 Books read by User 276709:
- The Sherbrooke Bride (Bride Trilogy (Paperback)) (Rating: 10)

📘 Top recommendations from similar users (User–User CF):
- Now You See Her (avg rating: 10.00)
- Divine Secrets of the Ya-Ya Sisterhood: A Novel (avg rating: 9.50)
- Hellion Bride (Bride Trilogy (Paperback)) (avg rating: 9.00)
- Deep Waters (avg rating: 9.00)
- Calypso Magic (avg rating: 9.00)
- A Cup of Comfort: Stories That Warm Your Heart, Lift Your Spirit, and Enrich Your Life (avg rating: 9.00)
- The Lottery Winner : Alvirah And Willy Stories (avg rating: 9.00)
- The Heiress Bride (Bride Trilogy (Paperback)) (avg rating: 9.00)
- The Wild Baron (avg rating: 9.00)
- The World's Dumbest Criminals (avg rating: 9.00)

📙 Books similar to what the user has read (Item–Item CF):
- The Heiress Bride (Bride Trilogy (Paperback)) (avg rating: 3.66)
- Hellion Bride (Bride Trilogy (Paperback)) (avg rating: 1.56)
- Someday My Prince (av